In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import copy
import seaborn as sns

In [2]:
# Add smr veg. weight band to raster

def smr_weights_raster(image, em_file) :
    '''Calculates the spectral mixture residual (smr) weights for each pixel
    in an image based on input endmember file.
    
    image: (string) path to raster 
    em_file: (string) path to .csv with reflectance values for each endmember
    
    returns: array with original image bands + endmembers'''
       

    # ATTENTION, MAKE SURE ENDMEMBER FILE IS ACCESSIBLE
    
    endmembers = endmember_file
    
    with rio.open(image) as src:
        raster_arr = np.array(src.read())
        raster_arr = raster_arr/10000 #scale reflectance values
   

    w = 1 # sum constraint weight
    D = raster_arr # numpy-nD array
    d = np.reshape(D, [D.shape[0], D.shape[1]*D.shape[2]]) # dimensions
    G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
    
    wavelength = G[:,0]
    G = G[:, 1:G.shape[1]].astype(np.float64)

    d_constraint = np.array(w*np.ones(d.shape[1]))
    G_constraint = np.array(w*np.ones(G.shape[1]))

    d = np.vstack([d, d_constraint])
    G = np.vstack([G, G_constraint])

    mixtureResidual_alt = d-(G.dot(np.linalg.inv(G.transpose().dot(G)).dot(G.transpose()))).dot(d)
    mixtureResidual_alt_reShp = np.reshape(mixtureResidual_alt,[d.shape[0],D.shape[1],D.shape[2]])
    
    M = np.linalg.inv(G.transpose().dot(G)).dot(G.transpose().dot(d))
    M_reShp = np.reshape(M,[M.shape[0], D.shape[1], D.shape[2]])
    
    w1 = M_reShp[0]
    w2 = M_reShp[1]
    
    em_ratio = np.divide(w1,\
                         w1+w2,\
                         out=np.zeros_like(w1),\
                         where=(w1+w2)!=0)
    
    #Reshape em_ratio to enable stacking with original 3D img array
    
    em_ratio_rs = em_ratio.reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    
    stacked_arr = np.vstack([raster_arr, em_ratio_rs])
    
    # Save stacked array as raster
    with rio.open(image) as src:
        kwargs = src.meta
        band_ct = stacked_arr.shape[0]
        kwargs.update(dtype=rio.float32, count=band_ct)
        
        with rio.open(str(os.path.split(image)[0])+'/SMR_'+str(os.path.basename(image)), 'w', **kwargs) as dst:
            for b in range(stacked_arr.shape[0]):
                dst.write_band(b+1, stacked_arr[b].astype(rio.float32))

In [8]:
# Add smr veg. weight AND VI bands to raster

def smr_vis_raster(image, endmember_file) :
    '''Calculates the spectral mixture residual (smr) weights for each pixel
    in an image based on input endmember file.
    
    image: (string) path to raster 
    em_file: (string) path to .csv with reflectance values for each endmember
    
    returns: array with original image bands + endmembers'''
       

    # ATTENTION, MAKE SURE ENDMEMBER FILE IS ACCESSIBLE
    
    endmembers = endmember_file
    
    with rio.open(image) as src:
        raster_arr = np.array(src.read())
        raster_arr = raster_arr/10000 #scale reflectance values
   

    w = 1 # sum constraint weight
    D = raster_arr # numpy-nD array
    d = np.reshape(D, [D.shape[0], D.shape[1]*D.shape[2]]) # dimensions
    G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
    
    wavelength = G[:,0]
    G = G[:, 1:G.shape[1]].astype(np.float64)

    d_constraint = np.array(w*np.ones(d.shape[1]))
    G_constraint = np.array(w*np.ones(G.shape[1]))

    d = np.vstack([d, d_constraint])
    G = np.vstack([G, G_constraint])

    mixtureResidual_alt = d-(G.dot(np.linalg.inv(G.transpose().dot(G)).dot(G.transpose()))).dot(d)
    mixtureResidual_alt_reShp = np.reshape(mixtureResidual_alt,[d.shape[0],D.shape[1],D.shape[2]])
    
    M = np.linalg.inv(G.transpose().dot(G)).dot(G.transpose().dot(d))
    M_reShp = np.reshape(M,[M.shape[0], D.shape[1], D.shape[2]])
    
    w1 = M_reShp[0]
    w2 = M_reShp[1]
    
    em_ratio = np.divide(w1,\
                         w1+w2,\
                         out=np.zeros_like(w1),\
                         where=(w1+w2)!=0)
    
    #Reshape em_ratio to enable stacking with original 3D img array
    
    veg_band = em_ratio.reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    
    
    # Add arrays (bands) with per-pixel VIs
    
    blue = raster_arr[0]
    green = raster_arr[1]
    red = raster_arr[2]
    nir = raster_arr[3]
    
    # MSAVI with SMR
    
    # Matrix of ones
    L_ones = np.ones(raster_arr[0].shape) 
    
    # Calculate L
    
    L = L_ones - veg_band 
    
    # Experimental MSAVI
    
    np.seterr(divide='ignore', invalid='ignore')
    
    savi_exp = ((nir - red) / (nir + red + L)) * (1 + L)
    savi = (1.5*(nir-red)/(nir+red+0.5)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    msavi = ((2*(nir) + 1 - np.sqrt((2*nir + 1)*2 - 8*(nir - red)))/2).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    
    evi = (2.5*(nir-red)/(nir+6*red-7.5*blue+1)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    arvi=((nir-(2*red-blue))/(nir+(2*red-blue))).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    gri = (green/red).reshape(1,em_ratio.shape[0],em_ratio.shape[1])

    ndvi = ((nir - red)/(nir + red)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    nir_blue = ((nir-blue)/(nir+blue)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    nir_green = ((nir-green)/(nir+green)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    red_blue = ((red-blue)/(red+blue)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    green_red = ((green-red)/(green+red)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    green_blue = ((green-blue)/(green+blue)).reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    
    stacked_arr = np.vstack([raster_arr,
                             veg_band,
                             savi_exp,
                             savi,
                             msavi,
                             evi,
                             arvi,
                             gri,
                             ndvi,
                             nir_blue,
                             nir_green,
                             red_blue,
                             green_red,
                             green_blue
                             ])
    
    # Save stacked array as raster
    with rio.open(image) as src:
        kwargs = src.meta
        band_ct = stacked_arr.shape[0]
        kwargs.update(dtype=rio.float32, count=band_ct)
        
        with rio.open(str(os.path.split(image)[0])+'/SMR_allVIs_'+str(os.path.basename(image)), 'w', **kwargs) as dst:
            for b in range(stacked_arr.shape[0]):
                dst.write_band(b+1, stacked_arr[b].astype(rio.float32))

In [9]:
# Test raster stacking with SS images from 2020

ss_2020 = glob.glob('../data/images/2020/SkySat/clipped_2020/20*.tif')
ss_2020

['../data/images/2020/SkySat/clipped_2020/20200618_160058_ssc3d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200625_154704_ssc12d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200812_153924_ssc3d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200805_154044_ssc13d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200710_155028_ssc3d2_0012_analytic_SR_corg_cor_clipped.tif']

In [10]:
em_file = '../data/endmembers_msi_noSwir.csv'
em_file

'../data/endmembers_msi_noSwir.csv'

In [11]:
for i in np.arange(len(ss_2020)):
    smr_vis_raster(ss_2020[i], em_file)

/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and dif

In [12]:
ss_2021 = glob.glob('../data/images/2021/SkySat/clipped_2021/20*.tif')
ss_2021

['../data/images/2021/SkySat/clipped_2021/20210809_185329_ssc9d2_0014_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210816_171007_ssc18d2_0015_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210707_150122_ssc19d2_0016_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210726_160338_ssc1d2_0013_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210802_184019_ssc10d2_0005_ar_clipped.tif']

In [16]:
for i in np.arange(len(ss_2021)):
    smr_vis_raster(ss_2021[i], em_file)

/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and dif

In [17]:
ss_2022 = glob.glob('../data/images/2022/SkySat/clipped_2022/*tif')
ss_2022

['../data/images/2022/SkySat/clipped_2022/20220622_160829_ssc15d2_0014_analytic_SR_corg_clipped.tif',
 '../data/images/2022/SkySat/clipped_2022/20220720_153112_ssc4d2_0013_analytic_SR_clipped_clipped.tif',
 '../data/images/2022/SkySat/clipped_2022/20220707_151445_ssc4d2_0006_analytic_SR_clipped_clipped.tif']

In [18]:
for i in np.arange(len(ss_2022)):
     smr_vis_raster(ss_2022[i], em_file)

/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_6540/2401207754.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and dif